## Optimization code for PV microgrid:

In [1]:
### Initialization:

using JuMP, Gurobi, MATLAB


In [2]:
for my_int = 1:96

    #----------------------------------------------------------------------------------------------------
    ### Solver Definition:
    
    m = Model(solver=GurobiSolver(Presolve=1,TimeLimit=900))
        
    #----------------------------------------------------------------------------------------------------
    ### Loading Optimization Parameters:

    mat"generate_data($my_int)"
    
    vars = read_matfile("../../Data/Generated Data/5 - Optimization/scenarios/scen_"*dec(my_int)*".mat")

    interval  = Int64(jvalue(vars["interval"]))  # Current Interval (in day, for which we are optimizing)

    N_EMS   = Int64(jvalue(vars["N_EMS"]))   # num of remaining EMS   intervals (opt. executions)
    N_intra = Int64(jvalue(vars["N_intra"])) # num of remaining Intra intervals (opt. resolution)
    N_dies  = Int64(jvalue(vars["N_dies"]))  # num of diesel generators
    N_scen  = Int64(jvalue(vars["N_scen"]))  # num of different scenations
    N_firm  = Int64(jvalue(vars["N_firm"]))  # num of FIRM Margin Constraint Intervals

    # Upper and Lower bounds:
    P_bat_min = jvalue(vars["P_bat_min"])
    P_bat_max = jvalue(vars["P_bat_max"])
    eta_bat  = jvalue(vars["eta_bat"])
    SOC_min  = jvalue(vars["SOC_min"])
    SOC_max  = jvalue(vars["SOC_max"])
    SOC_init = jvalue(vars["SOC_init"])
    Cap_bat  = jvalue(vars["Cap_bat"])
    P_dies_max = jvalue(vars["P_dies_max"])
    P_dies_min = jvalue(vars["P_dies_min"])
    P_PV_inst = jvalue(vars["P_PV_inst"])

    delta_t = jvalue(vars["delta_t"]) # 1 min

    # f_min = -100000
    f_min = jvalue(vars["f_min"])
    theta_i = jvalue(vars["theta_i"])
    theta_d = jvalue(vars["theta_d"])
    theta_b = jvalue(vars["theta_b"])
    theta_p = jvalue(vars["theta_p"])

    marge_dies = jvalue(vars["marge_dies"])

    L_PV = jvalue(vars["L_PV"]) # generated PV scenarios
    L_C  = jvalue(vars["L_C"]) # generated load scenarios

    # Defining iterators:
    ts  = 1:N_EMS
    ps  = 1:N_intra
    ds  = 1:N_dies
    ss  = 1:N_scen
    m1s = 1:N_firm
    m2s = (N_firm+1):N_EMS
    
    #----------------------------------------------------------------------------------------------------
    ### Variable definitions:

    @variables m begin

        # Battery variables:
        P_bat_min <= P_bat_set[t=ts,p=ps]  <= P_bat_max # Battery setpoint for time t,p (decision var.)
        0 <= P_bat_cha[t=ts,p=ps] <= P_bat_max
        0 <= P_bat_dis[t=ts,p=ps] <= P_bat_max
        X_bat[t=ts,p=ps], Bin                           # Operating mode indicator (decision var.)
                                                        # -> P_bat_set = 2*(X_bat-0.5)*P_bat_abs
        SOC_min <= SOC_bat[t=ts,p=ps] <= SOC_max        # Obtained SOC for time t,p (solution var.)

        # PV Plant:
        0 <= P_PV_set[t=ts,p=ps]  <= P_PV_inst          # PV setpoint variable (decision var.)
        0 <= P_PV[t=ts,p=ps,s=ss] <= L_PV[t,p,s]        # Actual PV generation (solution var.)

        # Diesel Generators:
        ON_dies[t=ts,d=ds], Bin                         # Dies. connectin setpoint (decision var.)
        0 <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max  # Actual dies. generation  (solution var.)
        # P_dies_min <= P_dies[t=ts,p=ps,s=ss,d=ds] <= P_dies_max # Actual dies. generation  (solution var.)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Variable Initialization (from previous optimization)
    
    if my_int>1

        vars = read_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(my_int-1)*".mat")

        P_bat_set_pre = jvalue(vars["P_bat_set"])[2:N_EMS,ps]
        SOC_bat_pre   = jvalue(vars["SOC_bat"])[2:N_EMS,ps]
        P_PV_set_pre  = jvalue(vars["P_PV_set"])[2:N_EMS,ps]
        P_PV_pre      = jvalue(vars["P_PV"])[2:N_EMS,ps,ss]
        ON_dies_pre   = jvalue(vars["ON_dies"])[2:N_EMS,ds]
        P_dies_pre    = jvalue(vars["P_dies"])[2:N_EMS,ps,ss,ds]
        P_bat_cha_pre = jvalue(vars["P_bat_cha"])[2:N_EMS,ps]
        P_bat_dis_pre = jvalue(vars["P_bat_dis"])[2:N_EMS,ps]
        X_bat_pre     = jvalue(vars["X_bat"])[2:N_EMS,ps]

        setvalue(P_bat_set[1:(N_EMS-1),ps],P_bat_set_pre)
        setvalue(SOC_bat[1:(N_EMS-1),ps],SOC_bat_pre)
        setvalue(P_PV_set[1:(N_EMS-1),ps],P_PV_set_pre)
        setvalue(P_PV[1:(N_EMS-1),ps,ss],P_PV_pre)
        setvalue(ON_dies[1:(N_EMS-1),ds],ON_dies_pre)
        setvalue(P_dies[1:(N_EMS-1),ps,ss,ds],P_dies_pre)
        setvalue(P_bat_cha[1:(N_EMS-1),ps],P_bat_cha_pre)
        setvalue(P_bat_dis[1:(N_EMS-1),ps],P_bat_dis_pre)
        setvalue(X_bat[1:(N_EMS-1),ps],X_bat_pre)

    end
    
    #----------------------------------------------------------------------------------------------------
    ### Constraints introduction:
    
    @constraints m begin

        # Constraints:
        #
        # A1 - Diesel power limitation
        # A2 - Diesel power limitation
        # B1 - Power balance equation
        # C1 - PV setpoint limitation
        # D1 - Bat. setpoint equation
        # D2 - Charging    mode const.
        # D3 - Discharging mode const.
        # E1 - FIRM    Diesel reserve margin (separated over scenarios)
        # E2 - RELAXED Diesel reserve margin (mean over scenarios)
        # F1 - 
        # F2 - SOC limitations
        # F3 - 
        # G1 - Min frequency limitation

        A1[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] <= ON_dies[t,d]*(P_dies_max)
        A2[t=ts,p=ps,s=ss,d=ds], P_dies[t,p,s,d] >= ON_dies[t,d]*(P_dies_min)

        B1[t=ts,p=ps,s=ss], (P_PV[t,p,s] + sum(P_dies[t,p,s,d] for d=ds) + P_bat_set[t,p] - L_C[t,p,s]) == 0

        C1[t=ts,p=ps,s=ss],  P_PV[t,p,s] <= P_PV_set[t,p]

        D1[t=ts,p=ps], P_bat_set[t,p] == P_bat_cha[t,p] - P_bat_dis[t,p]
        D2[t=ts,p=ps], P_bat_cha[t,p] <= P_bat_max *    X_bat[t,p]
        D3[t=ts,p=ps], P_bat_dis[t,p] <= P_bat_max * (1-X_bat[t,p])

        # E1[t=ts,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E1[t=m1s,p=ps,s=ss],  sum(P_dies[t,p,s,d] for d=ds) <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies
        E2[t=m2s,p=ps],  sum(P_dies[t,p,s,d] for d=ds,s=ss)/N_scen <= sum((ON_dies[t,d]*P_dies_max) for d=ds) - marge_dies

        F1[t=1,p=1],               SOC_bat[t,p] == SOC_init             - P_bat_set[t,p]*delta_t/Cap_bat
        F2[t=2:N_EMS,p=1],         SOC_bat[t,p] == SOC_bat[t-1,N_intra] - P_bat_set[t,p]*delta_t/Cap_bat
        F3[t=1:N_EMS,p=2:N_intra], SOC_bat[t,p] == SOC_bat[t,p-1]       - P_bat_set[t,p]*delta_t/Cap_bat

        G1[t=ts,p=ps], f_min <= theta_i + theta_d*sum(ON_dies[t,d] for d=ds) + theta_b*P_bat_set[t,p] + theta_p*P_PV_set[t,p]

    end

    #----------------------------------------------------------------------------------------------------
    ### Objective function definition:
    
    @objective(m, Max, sum((sum(P_PV[t,p,s] for s=ss)-N_scen*(1-eta_bat)*(P_bat_cha[t,p]+P_bat_dis[t,p])) for t=ts,p=ps));
    
    #----------------------------------------------------------------------------------------------------
    ### Solving Optimization:
    status = solve(m)

    #----------------------------------------------------------------------------------------------------
    ### Saving Data:
    
    write_matfile("../../Data/Generated Data/5 - Optimization/solutions/sol_"*dec(interval)*".mat"; 
        status    = string(status),
        interval  = interval,
        P_bat_set = getvalue(P_bat_set[:,:]), 
        SOC_bat   = getvalue(SOC_bat[:,:]),
        P_PV_set  = getvalue(P_PV_set[:,:]),
        P_PV      = getvalue(P_PV[:,:,:]),
        ON_dies   = getvalue(ON_dies[:,:]),
        P_dies    = getvalue(P_dies[:,:,:,:]),
        P_bat_cha = getvalue(P_bat_cha[:,:]),
        P_bat_dis = getvalue(P_bat_dis[:,:]),
        X_bat     = getvalue(X_bat[:,:]))
    
    # IJulia.clear_output(wait=false)
    println("Current progress: "*dec(my_int)*"/144")
    
    if string(status)=="Infeasible"
        break
    end
    
end

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]
Presolve removed 3664 rows and 4388 columns
Presolve time: 1.15s
Presolved: 192896 rows, 99004 columns, 595970 nonzeros
Variable types: 96412 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   32035    1.7260285e+07   1.053072e+06   0.000000e+00      5s
   57387    1.7220911e+07   7.816847e+05   0.000000e+00     10s
   74013    1.7189386e+07   6.507450e+05   0.000000e+00     15s
   93278    1.7150267e+07   5.136486e+05   0.000000e+00     20s
  115914    1.7071575e+07   3.913356e+05   0.000000e+00     25s
  141226    1.6873547e+07   2.630093e+05   0.000000e+00     30s
  162433  

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 105.041210800

Presolve removed 3665 rows and 4389 columns
Presolve time: 1.28s
Presolved: 192895 rows, 99003 columns, 595974 nonzeros
Variable types: 96411 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   27545    1.7251725e+07   1.205801e+06   0.000000e+00      5s
   58923    1.7193593e+07   8.607677e+05   0.000000e+00     10s
   85059    1.7153143e+07   5.882570e+05   0.000000e+00     15s
  106515    1.7098323e+07   4.622223e+05   0.000000e+00     20s
  131716    1.6977818e+07   3.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 552.850645367

Presolve removed 3664 rows and 4388 columns
Presolve time: 1.22s
Presolved: 192896 rows, 99004 columns, 595980 nonzeros
Variable types: 96412 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   27397    1.7146826e+07   1.213163e+06   0.000000e+00      5s
   59110    1.7092519e+07   7.769147e+05   0.000000e+00     10s
   84618    1.7049259e+07   6.067823e+05   0.000000e+00     15s
  106348    1.6996882e+07   4.596562e+05   0.000000e+00     20s
  129133    1.6886506e+07   3.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 95.771387916

Presolve removed 3686 rows and 4410 columns
Presolve time: 1.26s
Presolved: 192874 rows, 98982 columns, 595926 nonzeros
Variable types: 96390 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   24098    1.7413510e+07   1.241342e+06   0.000000e+00      5s
   55674    1.7367624e+07   7.879697e+05   0.000000e+00     10s
   82549    1.7321535e+07   6.007548e+05   0.000000e+00     15s
  103778    1.7279209e+07   4.608242e+05   0.000000e+00     20s
  128626    1.7149869e+07   3.0

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 537.174666622

Presolve removed 3684 rows and 4408 columns
Presolve time: 1.19s
Presolved: 192876 rows, 98984 columns, 595936 nonzeros
Variable types: 96392 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   24251    1.7210512e+07   1.256928e+06   0.000000e+00      5s
   56713    1.7151086e+07   8.199127e+05   0.000000e+00     10s
   82481    1.7105601e+07   5.712282e+05   0.000000e+00     15s
  102339    1.7060693e+07   4.989872e+05   0.000000e+00     20s
  125791    1.6954875e+07   3.

...

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [8e-01, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 353.046030535

Presolve removed 3682 rows and 4406 columns
Presolve time: 1.27s
Presolved: 192878 rows, 98986 columns, 595939 nonzeros
Variable types: 96394 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0450823e+07   1.031672e+07   0.000000e+00      6s
   45594    1.7277847e+07   9.026286e+05   0.000000e+00     10s
   72502    1.7234383e+07   6.574861e+05   0.000000e+00     15s
   93639    1.7194585e+07   5.527479e+05   0.000000e+00     20s
  114304    1.7131091e+07   3.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [7e-01, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 271.011539502

Presolve removed 3691 rows and 4415 columns
Presolve time: 1.40s
Presolved: 192869 rows, 98977 columns, 595915 nonzeros
Variable types: 96385 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0443204e+07   1.034673e+07   0.000000e+00      6s
   44633    1.7348518e+07   9.440673e+05   0.000000e+00     10s
   73180    1.7292976e+07   6.723189e+05   0.000000e+00     15s
   94582    1.7246452e+07   5.302940e+05   0.000000e+00     20s
  114968    1.7191673e+07   3.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [1e+00, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 850.488007008

Presolve removed 3694 rows and 4418 columns
Presolve time: 1.24s
Presolved: 192866 rows, 98974 columns, 595909 nonzeros
Variable types: 96382 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0128304e+07   1.023516e+07   0.000000e+00      6s
   46651    1.7181508e+07   9.277880e+05   0.000000e+00     10s
   73068    1.7137044e+07   6.950329e+05   0.000000e+00     15s
   93494    1.7099788e+07   5.622683e+05   0.000000e+00     20s
  112496    1.7045237e+07   4.

Academic license - for non-commercial use only
Optimize a model with 196560 rows, 103392 columns and 607679 nonzeros
Variable types: 100800 continuous, 2592 integer (2592 binary)
Coefficient statistics:
  Matrix range     [8e-08, 2e+03]
  Objective range  [3e-01, 1e+00]
  Bounds range     [2e-01, 2e+04]
  RHS range        [3e-01, 1e+04]

MIP start did not produce a new incumbent solution
MIP start violates constraint R172800 by 975.088841333

Presolve removed 3689 rows and 4413 columns
Presolve time: 1.32s
Presolved: 192871 rows, 98979 columns, 595925 nonzeros
Variable types: 96387 continuous, 2592 integer (2592 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0431036e+07   1.028225e+07   0.000000e+00      5s
   46185    1.7331307e+07   9.178742e+05   0.000000e+00     10s
   72211    1.7288906e+07   6.648649e+05   0.000000e+00     15s
   93021    1.7245483e+07   5.283401e+05   0.000000e+00     20s
  113073    1.7185171e+07   4.

Excessive output truncated after 524632 bytes.